In [1]:
storageLocation = "s3://dimajix-training/data/twitter-sample/00.json"

In [17]:
from pyspark.sql.functions import *

In [16]:
twitter = spark.read.json(storageLocation)
twitter.limit(5).toPandas()

,contributors,coordinates,created_at,delete,entities,extended_entities,favorite_count,favorited,filter_level,geo,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Fri Jul 29 08:00:00 +0000 2016,None,"([], None, [], [], [])",None,0,False,low,None,...,NaN,None,0,False,None,"<a href=""https://github.com/mispy/twitter_eboo...",Carrots coming in clutch with two Christmases.,1469779200658,False,"(False, Thu May 26 02:35:14 +0000 2016, False,..."
1,None,None,Fri Jul 29 08:00:00 +0000 2016,None,"([], None, [], [], [])",None,0,False,low,None,...,NaN,None,0,False,None,"<a href=""http://twitter.com/download/iphone"" r...",明日朝から夕方暇な方いらっしゃいませんかあ〜あ〜,1469779200662,False,"(False, Wed Apr 27 11:58:49 +0000 2016, True, ..."
2,None,None,Fri Jul 29 08:00:00 +0000 2016,None,"([], None, [], [(twitter.com/Bangin_is_15/s…, ...",None,0,False,low,None,...,7.589258e+17,758925763219300353,0,False,None,"<a href=""http://ifttt.com"" rel=""nofollow"">IFTT...",\76trdf\n— ゆぐどらしる(՞ةڼ◔) (Bangin_is_15) July 29...,1469779200666,False,"(False, Fri Jun 12 18:17:46 +0000 2015, True, ..."
3,None,None,Fri Jul 29 08:00:00 +0000 2016,None,"([], None, [], [], [])",None,0,False,low,None,...,NaN,None,0,False,None,"<a href=""http://twitter.com/download/iphone"" r...",Tシャツ諦めようと思ってたのに…,1469779200666,False,"(False, Tue Mar 15 14:06:45 +0000 2016, False,..."
4,None,None,Fri Jul 29 08:00:00 +0000 2016,None,"([([109, 114], عاجل), ([115, 124], السعودية)],...",None,0,False,low,None,...,NaN,None,0,False,None,"<a href=""http://twitterfeed.com"" rel=""nofollow...",تراجع أسعار المستهلكين في اليابان للشهر الرابع...,1469779200661,False,"(False, Thu Jun 27 07:28:44 +0000 2013, True, ..."


In [3]:
twitter.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- delete: struct (nullable = true)
 |    |-- status: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- user_id: long (nullable = true)
 |    |    |-- user_id_str: string (nullable = true)
 |    |-- timestamp_ms: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |  

# 1 Accessing Elements

In [11]:
result = twitter \
    .filter(twitter["geo"].isNotNull()) \
    .select(
        twitter["geo"]
    )
result.limit(5).toPandas()

,geo
0,"([51.826199, 4.62477], Point)"
1,"([51.6301173, 0.81552029], Point)"
2,"([36.19877126, 29.63925128], Point)"
3,"([13.28694, 100.929589], Point)"
4,"([30.32469225, 120.05983768], Point)"


In [12]:
result.printSchema()

root
 |-- geo: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)



## 2.1 Accessing nested entries

In [13]:
result = twitter \
    .filter(twitter["geo.coordinates"].isNotNull()) \
    .select(
        twitter["geo.coordinates"]
    )
result.limit(5).toPandas()

,coordinates
0,"[51.826199, 4.62477]"
1,"[51.6301173, 0.81552029]"
2,"[36.19877126, 29.63925128]"
3,"[13.28694, 100.929589]"
4,"[30.32469225, 120.05983768]"


In [14]:
result.printSchema()

root
 |-- coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)



## 2.2 Accessing Array Entries

In [9]:
result = twitter \
    .filter(twitter["geo.coordinates"].isNotNull()) \
    .select(
        twitter["geo.coordinates"][0],
        twitter["geo.coordinates"][1]
    )
result.limit(5).toPandas()

,geo.coordinates AS `coordinates`[0],geo.coordinates AS `coordinates`[1]
0,51.826199,4.624770
1,51.630117,0.815520
2,36.198771,29.639251
3,13.286940,100.929589
4,30.324692,120.059838


In [10]:
result.printSchema()

root
 |-- geo.coordinates AS `coordinates`[0]: double (nullable = true)
 |-- geo.coordinates AS `coordinates`[1]: double (nullable = true)



# 3 Exploding Entries

In [23]:
result = twitter \
    .select(
        twitter["id"],
        twitter["created_at"],
        explode(twitter["entities.hashtags"]).alias("hashtags")
    )
result.limit(5).toPandas()

,id,created_at,hashtags
0,758935090906882049,Fri Jul 29 08:00:00 +0000 2016,"([109, 114], عاجل)"
1,758935090906882049,Fri Jul 29 08:00:00 +0000 2016,"([115, 124], السعودية)"
2,758935090911031296,Fri Jul 29 08:00:00 +0000 2016,"([52, 62], KCAMexico)"
3,758935090911031296,Fri Jul 29 08:00:00 +0000 2016,"([63, 86], ValentinaZenereVillana)"
4,758935090902687744,Fri Jul 29 08:00:00 +0000 2016,"([27, 36], security)"


In [22]:
result = twitter \
    .select(
        twitter["id"],
        twitter["created_at"],
        explode(twitter["entities.hashtags.text"]).alias("hashtags")
    )
result.limit(5).toPandas()

,id,created_at,hashtags
0,758935090906882049,Fri Jul 29 08:00:00 +0000 2016,عاجل
1,758935090906882049,Fri Jul 29 08:00:00 +0000 2016,السعودية
2,758935090911031296,Fri Jul 29 08:00:00 +0000 2016,KCAMexico
3,758935090911031296,Fri Jul 29 08:00:00 +0000 2016,ValentinaZenereVillana
4,758935090902687744,Fri Jul 29 08:00:00 +0000 2016,security
